In [1]:
import sys
sys.path.append('C:\\Users\\gabri\\my_projects\\stock_analysis')

import plotly.express as px
import pandas as pd
from stock import yfScraper, StockAnalyzer, TickerScraper
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import json
import re
    

In [2]:
def extract_highest_amount(amount_str):
    numbers = re.findall(r'\d+', amount_str.replace(',', ''))
    if not numbers:
        return None
    return int(numbers[-1])

In [3]:
path = r'C:\Users\gabri\my_projects\stock_analysis\data\senate_stock_tracker_2024-06-09.json'
with open(path, 'r') as file:
    data = json.load(file)

df = pd.json_normalize(data)
df['highest_amount'] = df['amount'].apply(extract_highest_amount)
df['in_out'] = df['type'].replace({'purchase':'in', 'sale_partial':'out', 'sale_full':'out', 'exchange':'out', 'sale':'out'})
df.columns

Index(['disclosure_year', 'disclosure_date', 'transaction_date', 'owner',
       'ticker', 'asset_description', 'type', 'amount', 'representative',
       'district', 'state', 'ptr_link', 'cap_gains_over_200_usd', 'industry',
       'sector', 'party', 'highest_amount', 'in_out'],
      dtype='object')

In [4]:
df1 = df.copy()
df1 = df1[['ticker', 'highest_amount']]

mean_highest_amount = df1.groupby('ticker').mean('highest_amount')
ticker_counts = df1['ticker'].value_counts()
result = pd.DataFrame({
    'mean_highest_amount': mean_highest_amount['highest_amount'],
    'count': ticker_counts
}).reset_index()
result = result.sort_values(by='count', ascending=False)

fig = px.bar(result, x='ticker', y='count', title='Ticker Counts', color ='mean_highest_amount')
fig.show()

In [5]:
result = result[(result['count'] > 100) & (result['ticker'] != '--')]
result = result.sort_values(by='mean_highest_amount', ascending=False)
fig = px.bar(result, x='ticker', y='mean_highest_amount', title='Ticker Counts', color ='count')
fig.show()

In [17]:
scrape = yfScraper(r'C:\Users\gabri\my_projects\stock_analysis\ticker_attributes.json')
tickers = ['MSFT', 'NVDA']
df_p = scrape.get_stock_price_dfs(tickers)
stock_analysis = StockAnalyzer(scrape)
stock_analysis.line_plot(['MSFT', '^GSPC'])

File exists, successfully loaded attributes file: C:\Users\gabri\my_projects\stock_analysis\ticker_attributes.json


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\gabri\\my_projects\\stock_analysis\\data\\^GSPC_1d_1927-12-30_2024-06-16_processed.csv'